In [1]:
import scipy.io

In [3]:
mat_file_path = '/home/safin/MobileFaceNet_Pytorch/result/best_result.mat'
result = scipy.io.loadmat(mat_file_path)


In [5]:
fl = result['fl']
fr = result['fr']

In [6]:
import torch
l2dist = torch.nn.CosineSimilarity()

In [11]:
N = fl.shape[0]

In [19]:
step = 64
arr = []
for i in range(N//step + 1):
    cur_fl = torch.from_numpy(fl[i*step:(i+1)*step])
    cur_fr = torch.from_numpy(fr[i*step:(i+1)*step])
    arr += l2dist(cur_fl, cur_fr).numpy().tolist()
arr = np.asarray(arr)

In [35]:
from utils import KFold
import numpy as np

def k_fold_eval(dists, labels):
#     thresholds = np.arange(-1.0, 1.0, 0.001)
    thrNum = 10000
    thresholds = np.arange(-thrNum, thrNum + 1) * 1.0 / thrNum
    acc_arr = []
    for pairs in KFold(n=6000, n_folds=10):
        train_pairs, test_pairs = pairs
        t, _ = find_best(thresholds, dists[train_pairs], labels[train_pairs])
        acc_arr.append(eval_acc(t, dists[test_pairs], labels[test_pairs]))
    return np.mean(acc_arr), np.std(acc_arr)

def eval_acc(threshold, dists, labels):
    accuracy = ((dists > threshold) == labels).mean()
    return accuracy

def find_best(thresholds, dists, labels):
    best_threshold = best_acc = 0
    for threshold in thresholds:
        accuracy = eval_acc(threshold, dists, labels)
        if accuracy >= best_acc:
            best_acc = accuracy
            best_threshold = threshold
    return best_threshold, best_acc

In [44]:
def k_fold_eval(featureLs, featureRs, labels):
#     thresholds = np.arange(-1.0, 1.0, 0.001)
    thrNum = 10000
    thresholds = np.arange(-thrNum, thrNum + 1) * 1.0 / thrNum
    acc_arr = []
    for pairs in KFold(n=6000, n_folds=10):
        train_pairs, test_pairs = pairs
        mu = np.mean(np.concatenate((featureLs[train_pairs, :], featureRs[train_pairs, :]), 0), 0)
#         print(mu)
        mu = np.expand_dims(mu, 0)
        fLs = featureLs #- mu
        fRs = featureRs #- mu
        fLs = fLs / np.expand_dims(np.sqrt(np.sum(np.power(fLs, 2), 1)), 1)
        fRs = fRs / np.expand_dims(np.sqrt(np.sum(np.power(fRs, 2), 1)), 1)
        scores = np.sum(np.multiply(fLs, fRs), 1)
        t, _ = find_best(thresholds, scores[train_pairs], labels[train_pairs])
        acc_arr.append(eval_acc(t, scores[test_pairs], labels[test_pairs]))
    return np.mean(acc_arr), np.std(acc_arr)

In [28]:
labels = [1]*300 + [0]*300
labels = labels*10
labels = np.asarray(labels)

In [36]:
k_fold_eval(arr, labels)

(0.9916666666666666, 0.0036514837167010952)

In [32]:
thrNum = 10000
thresholds = np.arange(-thrNum, thrNum + 1) * 1.0 / thrNum

In [34]:
thresholds

array([-1.    , -0.9999, -0.9998, ...,  0.9998,  0.9999,  1.    ])

In [45]:
k_fold_eval(fl, fr, labels)

(0.9916666666666666, 0.0036514837167010952)